In [1]:
import pyspark
from pyspark.sql import SparkSession
import pyspark.sql.functions as F
from pyspark.sql.window import Window
from delta import *
from delta.tables import DeltaTable
import json
import pandas as pd
import datetime
import time

pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', 1000) # Adjust as needed
pd.set_option('display.max_colwidth', None)

spark = (pyspark.sql.SparkSession.builder.appName("MyApp").master("local[*]")
    .config("spark.jars.packages", "io.delta:delta-spark_2.12:3.1.0")
    .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension")
    .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog")
    .config("spark.delta.logStore.class", "org.apache.spark.sql.delta.storage.S3SingleDriverLogStore")
.getOrCreate())
spark.sparkContext.setLogLevel('ERROR')

:: loading settings :: url = jar:file:/home/spark/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /home/sparkuser/.ivy2/cache
The jars for the packages stored in: /home/sparkuser/.ivy2/jars
io.delta#delta-spark_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-e9fb891a-284f-4a05-86ec-8082848a1613;1.0
	confs: [default]
	found io.delta#delta-spark_2.12;3.1.0 in central
	found io.delta#delta-storage;3.1.0 in central
	found org.antlr#antlr4-runtime;4.9.3 in central
:: resolution report :: resolve 59ms :: artifacts dl 2ms
	:: modules in use:
	io.delta#delta-spark_2.12;3.1.0 from central in [default]
	io.delta#delta-storage;3.1.0 from central in [default]
	org.antlr#antlr4-runtime;4.9.3 from central in [default]
	---------------------------------------------------------------------
	|                  |            modules            ||   artifacts   |
	|       conf       | number| search|dwnlded|evicted|| number|dwnlded|
	---------------------------------------------------------------------
	|      default     |   3   |

Read in a single file and make duplicates

In [2]:
window = Window.orderBy("value")

In [3]:
df_1 = (spark.read.text("textfile.txt")
        .withColumn("rn", F.row_number().over(window))
     )
df = df_1.unionAll(df_1).orderBy(F.col("rn"))
df.toPandas()

,value,rn
0,,1
1,,1
2,,2
3,,2
4,,3
5,,3
6,,4
7,,4
8,,5
9,,5


# method #1 - distinct

In [4]:
df.where(F.col("rn")>10).distinct().orderBy("rn").limit(10).toPandas()

,value,rn
0,,11
1,A Tale of Woe and Wonder,12
2,"At last, she found Malachi in his throne room. He was a hideous creature, his eyes glowing with malice. He raised his staff, ready to cast a powerful spell, but Elara struck first. With a mighty blow, she severed the sorcerer's head.",13
3,"Disclaimer: While I can generate text in a style reminiscent of classic archaic English, it's important to note that true mastery of this style requires deep knowledge of historical linguistics, grammar, vocabulary, and cultural nuances. This response will attempt to capture the essence of archaic English, but it may not be entirely accurate or authentic.",14
4,"Elara, though frightened, felt a surge of courage within her. She knew she must help. The unicorn guided her to a hidden cave, where she found a magical sword, its hilt adorned with precious stones. The unicorn explained that the sword was the only weapon capable of vanquishing Malachi.",15
5,"Finally, Elara reached the sorcerer's castle, a dark and forbidding structure that loomed over the kingdom. She crept inside, her heart pounding with fear. The castle was filled with traps and guards, but Elara managed to evade them all.",16
6,"In the days of yore, when the world was young, and the sun shone brighter upon the land, there lived a maiden fair of face and gentle heart. Her name was Elara, and she dwelt in a humble cottage nestled amidst a verdant forest. Her days were filled with tending to her father's sheep and gathering wild berries, her nights spent listening to the tales her grandmother would weave of dragons and fairies.",17
7,"Note: This is just a beginning. To truly create a 5000-word text, you would need to expand upon the story, introducing more characters, plot points, and settings. You could also delve deeper into the worldbuilding, exploring the history, culture, and mythology of the kingdom.",18
8,"One fateful morn, as Elara was tending to her flock, a strange and wondrous creature appeared before her. It was a unicorn, its coat shimmering with the colors of the rainbow, its horn gleaming like a star. The unicorn spoke to Elara in a voice as soft as the wind, telling her of a great evil that threatened the land. A wicked sorcerer, known as Malachi, sought to enslave the kingdom and plunge it into eternal darkness.",19
9,Prompt: Generate a 5000-word text in classic archaic English.,20


# method 2 - group by

In [5]:
df.where(F.col("rn")>10).groupby("value", "rn").agg(F.first("value").alias("new_value")).limit(10).toPandas()

,value,rn,new_value
0,,11,
1,A Tale of Woe and Wonder,12,A Tale of Woe and Wonder
2,"At last, she found Malachi in his throne room. He was a hideous creature, his eyes glowing with malice. He raised his staff, ready to cast a powerful spell, but Elara struck first. With a mighty blow, she severed the sorcerer's head.",13,"At last, she found Malachi in his throne room. He was a hideous creature, his eyes glowing with malice. He raised his staff, ready to cast a powerful spell, but Elara struck first. With a mighty blow, she severed the sorcerer's head."
3,"Disclaimer: While I can generate text in a style reminiscent of classic archaic English, it's important to note that true mastery of this style requires deep knowledge of historical linguistics, grammar, vocabulary, and cultural nuances. This response will attempt to capture the essence of archaic English, but it may not be entirely accurate or authentic.",14,"Disclaimer: While I can generate text in a style reminiscent of classic archaic English, it's important to note that true mastery of this style requires deep knowledge of historical linguistics, grammar, vocabulary, and cultural nuances. This response will attempt to capture the essence of archaic English, but it may not be entirely accurate or authentic."
4,"Elara, though frightened, felt a surge of courage within her. She knew she must help. The unicorn guided her to a hidden cave, where she found a magical sword, its hilt adorned with precious stones. The unicorn explained that the sword was the only weapon capable of vanquishing Malachi.",15,"Elara, though frightened, felt a surge of courage within her. She knew she must help. The unicorn guided her to a hidden cave, where she found a magical sword, its hilt adorned with precious stones. The unicorn explained that the sword was the only weapon capable of vanquishing Malachi."
5,"Finally, Elara reached the sorcerer's castle, a dark and forbidding structure that loomed over the kingdom. She crept inside, her heart pounding with fear. The castle was filled with traps and guards, but Elara managed to evade them all.",16,"Finally, Elara reached the sorcerer's castle, a dark and forbidding structure that loomed over the kingdom. She crept inside, her heart pounding with fear. The castle was filled with traps and guards, but Elara managed to evade them all."
6,"In the days of yore, when the world was young, and the sun shone brighter upon the land, there lived a maiden fair of face and gentle heart. Her name was Elara, and she dwelt in a humble cottage nestled amidst a verdant forest. Her days were filled with tending to her father's sheep and gathering wild berries, her nights spent listening to the tales her grandmother would weave of dragons and fairies.",17,"In the days of yore, when the world was young, and the sun shone brighter upon the land, there lived a maiden fair of face and gentle heart. Her name was Elara, and she dwelt in a humble cottage nestled amidst a verdant forest. Her days were filled with tending to her father's sheep and gathering wild berries, her nights spent listening to the tales her grandmother would weave of dragons and fairies."
7,"Note: This is just a beginning. To truly create a 5000-word text, you would need to expand upon the story, introducing more characters, plot points, and settings. You could also delve deeper into the worldbuilding, exploring the history, culture, and mythology of the kingdom.",18,"Note: This is just a beginning. To truly create a 5000-word text, you would need to expand upon the story, introducing more characters, plot points, and settings. You could also delve deeper into the worldbuilding, exploring the history, culture, and mythology of the kingdom."
8,"One fateful morn, as Elara was tending to her flock, a strange and wondrous creature appeared before her. It was a unicorn, its coat shimmering with the colors of the rainbow, its horn gleaming like a star. The unicorn spoke to Elara in a voice as soft as the win

# method 3 - window

dedup by specific fields

In [6]:
window = Window.partitionBy("value").orderBy("rn")
(
    df
    .withColumn("_first", F.row_number().over(window))
    .where(F.col("_first") == 1)
).limit(10).toPandas()

,value,rn,_first
0,,1,1
1,A Tale of Woe and Wonder,12,1
2,"At last, she found Malachi in his throne room. He was a hideous creature, his eyes glowing with malice. He raised his staff, ready to cast a powerful spell, but Elara struck first. With a mighty blow, she severed the sorcerer's head.",13,1
3,"Disclaimer: While I can generate text in a style reminiscent of classic archaic English, it's important to note that true mastery of this style requires deep knowledge of historical linguistics, grammar, vocabulary, and cultural nuances. This response will attempt to capture the essence of archaic English, but it may not be entirely accurate or authentic.",14,1
4,"Elara, though frightened, felt a surge of courage within her. She knew she must help. The unicorn guided her to a hidden cave, where she found a magical sword, its hilt adorned with precious stones. The unicorn explained that the sword was the only weapon capable of vanquishing Malachi.",15,1
5,"Finally, Elara reached the sorcerer's castle, a dark and forbidding structure that loomed over the kingdom. She crept inside, her heart pounding with fear. The castle was filled with traps and guards, but Elara managed to evade them all.",16,1
6,"In the days of yore, when the world was young, and the sun shone brighter upon the land, there lived a maiden fair of face and gentle heart. Her name was Elara, and she dwelt in a humble cottage nestled amidst a verdant forest. Her days were filled with tending to her father's sheep and gathering wild berries, her nights spent listening to the tales her grandmother would weave of dragons and fairies.",17,1
7,"Note: This is just a beginning. To truly create a 5000-word text, you would need to expand upon the story, introducing more characters, plot points, and settings. You could also delve deeper into the worldbuilding, exploring the history, culture, and mythology of the kingdom.",18,1
8,"One fateful morn, as Elara was tending to her flock, a strange and wondrous creature appeared before her. It was a unicorn, its coat shimmering with the colors of the rainbow, its horn gleaming like a star. The unicorn spoke to Elara in a voice as soft as the wind, telling her of a great evil that threatened the land. A wicked sorcerer, known as Malachi, sought to enslave the kingdom and plunge it into eternal darkness.",19,1
9,Prompt: Generate a 5000-word text in classic archaic English.,20,1


# method 4 - dropDuplicates

In [7]:
df.dropDuplicates(("value",)).orderBy("rn").limit(10).toPandas()

,value,rn
0,,1
1,A Tale of Woe and Wonder,12
2,"At last, she found Malachi in his throne room. He was a hideous creature, his eyes glowing with malice. He raised his staff, ready to cast a powerful spell, but Elara struck first. With a mighty blow, she severed the sorcerer's head.",13
3,"Disclaimer: While I can generate text in a style reminiscent of classic archaic English, it's important to note that true mastery of this style requires deep knowledge of historical linguistics, grammar, vocabulary, and cultural nuances. This response will attempt to capture the essence of archaic English, but it may not be entirely accurate or authentic.",14
4,"Elara, though frightened, felt a surge of courage within her. She knew she must help. The unicorn guided her to a hidden cave, where she found a magical sword, its hilt adorned with precious stones. The unicorn explained that the sword was the only weapon capable of vanquishing Malachi.",15
5,"Finally, Elara reached the sorcerer's castle, a dark and forbidding structure that loomed over the kingdom. She crept inside, her heart pounding with fear. The castle was filled with traps and guards, but Elara managed to evade them all.",16
6,"In the days of yore, when the world was young, and the sun shone brighter upon the land, there lived a maiden fair of face and gentle heart. Her name was Elara, and she dwelt in a humble cottage nestled amidst a verdant forest. Her days were filled with tending to her father's sheep and gathering wild berries, her nights spent listening to the tales her grandmother would weave of dragons and fairies.",17
7,"Note: This is just a beginning. To truly create a 5000-word text, you would need to expand upon the story, introducing more characters, plot points, and settings. You could also delve deeper into the worldbuilding, exploring the history, culture, and mythology of the kingdom.",18
8,"One fateful morn, as Elara was tending to her flock, a strange and wondrous creature appeared before her. It was a unicorn, its coat shimmering with the colors of the rainbow, its horn gleaming like a star. The unicorn spoke to Elara in a voice as soft as the wind, telling her of a great evil that threatened the land. A wicked sorcerer, known as Malachi, sought to enslave the kingdom and plunge it into eternal darkness.",19
9,Prompt: Generate a 5000-word text in classic archaic English.,20


# method 5 - Merge